Сначала мы попробуем использовать простую вариационную форму с одним кубитом для решения проблемы, аналогичной оценке энергии основного состояния. В частности, нам дан случайный вектор вероятности $\mathbf{x}$ и и мы хотим определить возможную параметризацию для нашей вариационной формы с одним кубитом, чтобы она выводила распределение вероятностей, близкое к $\mathbf{x}$ (где близость определяется в терминах манхэттенского расстояния между двумя векторами вероятности).

In [2]:
import qiskit
import scipy
import qiskit_nature
from qiskit.algorithms.optimizers import COBYLA
from cmath import phase
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.extensions import Initialize
from qiskit.quantum_info import random_statevector, Statevector
import qiskit.algorithms.optimizers

In [3]:
import numpy as np
target_distr = np.random.rand(2) # Создали случайный вектор
# Теперь преобразуем случайный вектор в вектор вероятности (нормируем на 1)
target_distr /= sum(target_distr) #Получили \psi

In [4]:
def get_var_form(params): #Фукнция создаёт квантовую схему q c вариационной формой U3. Аргументы функции = аргументы формы
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

Теперь мы определяем целевую функцию, которая принимает в качестве входных данных список параметров вариационной формы и возвращает "манхетеннскую разность" между ... и начальным вектором вероятности

In [5]:
backend = Aer.get_backend('qasm_simulator') #подклюичили аналоговый симмулятор
NUM_SHOTS = 100000

def get_probability_distribution(counts): # Получим распределение вероятности (counts- словарь)
    output_distr = [v / NUM_SHOTS for v in counts.values()] #метод .values() вытащит только значения словаря, которые мы поделим на 10 000
    if len(output_distr) == 1:
        output_distr.append(1 - output_distr[0]) # чтобы не было вектора длины 1, мы зададим вектор длины 2 (1 0)
    return output_distr

def objective_function(params):
    # Получаем квантовую схему по заданным параметрам формы U3
    qc = get_var_form(params)
    # Выполните квантовую схему для получения распределения вероятностей, связанного с текущими параметрами
    t_qc = transpile(qc, backend) # транспилируем схему: делаем эквивалентные замены гейтов, чтобы уменьшить вер-ть ошибки
    qobj = assemble(t_qc, shots=NUM_SHOTS) # Эта функция сериализует SHOTS раз схему, для создания “экспериментов” Qobj. 
    result = backend.run(qobj).result().get_counts(t_qc) #Запускаем на сиуляторе backend qobj и получаем результаты
    #Получите значения для каждого измеренного состояния и преобразуйте эти значения в вектор вероятности
    output_distr = get_probability_distribution(dict(result)) # Получим распределение вероятности после псоле прохождения схемы \psi_{\theta}
    
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)]) # Посчитаем манхетеннское расстояние между векторами вероятности
    return cost

In [7]:

# Создайте начальные параметры (аргументы формы)
params = np.random.rand(3)
ret = scipy.optimize.minimize(method='COBYLA',options={'maxiter': 500, 'catol': 0.0001}, fun=objective_function, x0=params)

# Получите выходное распределение, используя оптимизированные параметры
rret = list(ret.values())
qc = get_var_form(rret[0])
t_qc = transpile(qc, backend)
qobj = assemble(t_qc, shots=NUM_SHOTS)
counts = backend.run(qobj).result().get_counts(t_qc)
output_distr = get_probability_distribution(counts)

print("Начальное распределение:", target_distr)
print("Оптимизированное распределение:", output_distr)
print("Манхеттенское расстояние между векторами распределения до оптимизации:", objective_function(params))
print("Манхеттенское расстояние между векторами распределения после оптимизации:", rret[-2])
print("Исходные параметры:", params)
print("Оптимизированные параметры:", rret[0])

Начальное распределение: [0.89423843 0.10576157]
Оптимизированное распределение: [0.10629, 0.89371]
Манхеттенское расстояние между векторами распределения до оптимизации: 0.21012313258268606
Манхеттенское расстояние между векторами распределения после оптимизации: 0.0016568674173138564
Исходные параметры: [0.04826692 0.92928105 0.23029384]
Оптимизированные параметры: [-0.66191622  2.10659885  0.39546556]
